In [1]:
from xml.etree.ElementTree import fromstring
from pyspark.sql import SparkSession
from Preprocessing.data_manipulation import DataPreparation
from Transformers.data_aggregation import AggregateData
from Transformers.impute_mean import ImputePrice
from Transformers.negative_sales import NegativeSales
from Transformers.logtransformer import Log
from pyspark.ml import Pipeline
from Transformers.scalar_na_filler import ScallerNAFiller
from Transformers.lagtransformer import Lags
from Transformers.test_train_split import Split
from pyspark.ml.feature import VectorAssembler, StringIndexer
from Evaluator.Mape import MAPE
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from Estimator.random_forest import RandomForest
from Estimator.XGBoost import XGBoost
from Transformers.antilogtransformer import AntiLog
import pickle

D:\Data_M5\calendar.csv
D:\Data_M5\sell_prices.csv
D:\Data_M5\train_data_modified.csv


In [2]:
# Creating a Spark Session
if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("project_spark") \
        .master("local[*]") \
        .config("spark.driver.memory", "12g")\
        .getOrCreate()

In [3]:
# df = Data Frame 
data = DataPreparation()
df_m5 = data.get_data()

In [4]:
df_m5.groupBy("store_id").count().show()

In [5]:
df_m5.groupBy('store_id').agg({"sell_price": "avg", "sales": "sum"} ).show()

In [6]:
df_m5 = data.filter_store(df_m5, "CA_1")
df_m5.show(5)

+--------+-----------+--------+--------------------+-------+------+--------+-----+----------+---------+----+-----+----+------------+------------+------------+------------+-------+-------+-------+----------+
|store_id|    item_id|wm_yr_wk|                  id|dept_id|cat_id|state_id|sales|      date|  weekday|wday|month|year|event_name_1|event_type_1|event_name_2|event_type_2|snap_CA|snap_TX|snap_WI|sell_price|
+--------+-----------+--------+--------------------+-------+------+--------+-----+----------+---------+----+-----+----+------------+------------+------------+------------+-------+-------+-------+----------+
|    CA_1|FOODS_1_001|   11535|FOODS_1_001_CA_1_...|FOODS_1| FOODS|      CA|    0|2015-09-26| Saturday|   1|    9|2015|        null|        null|        null|        null|      0|      0|      0|      2.24|
|    CA_1|FOODS_1_001|   11535|FOODS_1_001_CA_1_...|FOODS_1| FOODS|      CA|    0|2015-09-27|   Sunday|   2|    9|2015|        null|        null|        null|        null| 

In [8]:
df_m5.printSchema()

root
 |-- store_id: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- wm_yr_wk: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- cat_id: string (nullable = true)
 |-- state_id: string (nullable = true)
 |-- sales: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- weekday: string (nullable = true)
 |-- wday: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- event_name_1: string (nullable = true)
 |-- event_type_1: string (nullable = true)
 |-- event_name_2: string (nullable = true)
 |-- event_type_2: string (nullable = true)
 |-- snap_CA: integer (nullable = true)
 |-- snap_TX: integer (nullable = true)
 |-- snap_WI: integer (nullable = true)
 |-- sell_price: double (nullable = true)



In [9]:
df_m5.columns

['store_id',
 'item_id',
 'wm_yr_wk',
 'id',
 'dept_id',
 'cat_id',
 'state_id',
 'sales',
 'date',
 'weekday',
 'wday',
 'month',
 'year',
 'event_name_1',
 'event_type_1',
 'event_name_2',
 'event_type_2',
 'snap_CA',
 'snap_TX',
 'snap_WI',
 'sell_price']

# Initiating different Transformers #

In [10]:
imputeNegativePrice = ImputePrice()
negativeSales = NegativeSales(column="sales")
aggregate = AggregateData(columns=["store_id", "dept_id", "year", "month"],
                                expressions={"sales": "sum",
                                "sell_price": "avg",
                                "snap_WI": "sum"})

In [11]:
log_transform = Log(inputCols=["sales","sell_price"])

In [12]:
lag_feature_transform = Lags(lags=[1,2,3], target="sales", partitionBy=["store_id","dept_id"], orderBy=["year", "month"])

In [13]:
na_filler = ScallerNAFiller()

In [14]:
storeIndexer = StringIndexer(inputCol="store_id", outputCol="store_id_index")
yearIndexer = StringIndexer(inputCol="year", outputCol="year_index")

In [15]:
inputColumns = ['month',
        'sell_price',
        'snap_WI',
        'lag_1',
        'lag_2',
        'lag_3',
        'store_id_index',
        'year_index']
assembler = VectorAssembler(inputCols=inputColumns, 
                                    outputCol="features")

#  Preprocessing/FeatureEngineering Pipeline Complete #

In [16]:
transformed = Pipeline(stages=[imputeNegativePrice, negativeSales, aggregate, 
                       log_transform, lag_feature_transform, storeIndexer, yearIndexer, na_filler, assembler]).fit(df_m5).transform(df_m5)

In [17]:
transformed = transformed.cache()
transformed.count()

448

In [18]:
transformed.show()

+--------+---------+----+-----+------------------+-------+------------------+------------------+------------------+------------------+--------------+----------+--------------------+
|store_id|  dept_id|year|month|        sell_price|snap_WI|             sales|             lag_1|             lag_2|             lag_3|store_id_index|year_index|            features|
+--------+---------+----+-----+------------------+-------+------------------+------------------+------------------+------------------+--------------+----------+--------------------+
|    CA_1|HOBBIES_2|2011|    1| 0.955345645099547|      0|3.7612001156935624|               0.0|               0.0|               0.0|           0.0|       0.0|(8,[0,1],[1.0,0.9...|
|    CA_1|HOBBIES_2|2011|    2|0.9553456450995472|   1490| 5.877735781779639|3.7612001156935624|               0.0|               0.0|           0.0|       0.0|(8,[0,1,2,3],[2.0...|
|    CA_1|HOBBIES_2|2011|    3|0.9553456450995466|   1490| 6.423246963533519| 5.8777357817

In [19]:
transformed.printSchema()

root
 |-- store_id: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- sell_price: double (nullable = false)
 |-- snap_WI: long (nullable = true)
 |-- sales: double (nullable = false)
 |-- lag_1: double (nullable = false)
 |-- lag_2: double (nullable = false)
 |-- lag_3: double (nullable = false)
 |-- store_id_index: double (nullable = false)
 |-- year_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [20]:
antilogtransformer = AntiLog(inputCols=["sales"])

In [21]:
transformered = antilogtransformer.transform(transformed)
transformered.show()

+--------+---------+----+-----+------------------+-------+------------------+------------------+------------------+------------------+--------------+----------+--------------------+
|store_id|  dept_id|year|month|        sell_price|snap_WI|             sales|             lag_1|             lag_2|             lag_3|store_id_index|year_index|            features|
+--------+---------+----+-----+------------------+-------+------------------+------------------+------------------+------------------+--------------+----------+--------------------+
|    CA_1|HOBBIES_2|2011|    1| 0.955345645099547|      0|              43.0|               0.0|               0.0|               0.0|           0.0|       0.0|(8,[0,1],[1.0,0.9...|
|    CA_1|HOBBIES_2|2011|    2|0.9553456450995472|   1490| 356.9999999999999|3.7612001156935624|               0.0|               0.0|           0.0|       0.0|(8,[0,1,2,3],[2.0...|
|    CA_1|HOBBIES_2|2011|    3|0.9553456450995466|   1490| 615.9999999999998| 5.8777357817

# Random Forest Training #

In [17]:
spliting = DataPreparation()

In [18]:
train_df, test_df = spliting.train_test_split(transformed, 2015)

# RANDOM FOREST TRAINING #

In [19]:
rfModel = RandomForest(featuresCol="features", labelCol="sales").fit(train_df)

mape score: 0.014296464076371815
mape score:                                          
0.012363409278122416                                 
mape score:                                                                       
0.017994850354753577                                                              
mape score:                                                                       
0.012041132070446532                                                              
mape score:                                                                       
0.014064548120648633                                                              
mape score:                                                                       
0.011038194405677488                                                              
100%|██████████| 5/5 [12:25<00:00, 149.07s/trial, best loss: 0.011038194405677488]
{'maxBins': 39.0, 'maxDepth': 10.0, 'numTrees': 65.0, 'subsamplingRate': 0.9}


In [23]:
rfModel.save("rfmodel.sav")

In [25]:
rf_prediction = rfModel.transform(test_df)

In [26]:
rf_prediction.select(["store_id","year","month","sales","prediction"]).show(10)

+--------+----+-----+-----------------+-----------------+
|store_id|year|month|            sales|       prediction|
+--------+----+-----+-----------------+-----------------+
|    CA_1|2015|    1|7.065613363597717|7.544214479794464|
|    CA_1|2015|    2| 7.00033446027523|6.946529825313822|
|    CA_1|2015|    3|7.212294468500341| 6.62432624982336|
|    CA_1|2015|    4|7.355641102974253|7.329366636250969|
|    CA_1|2015|    5|7.315218389752975|7.366029644624244|
|    CA_1|2015|    6|7.097548850614793|7.343149957590786|
|    CA_1|2015|    7|7.123672785204607|7.369674197563143|
|    CA_1|2015|    8|7.136483208590247|7.412908428746474|
|    CA_1|2015|    9|6.954638864880987|7.380855663822053|
|    CA_1|2015|   10|7.500529485395295|6.657383137102154|
+--------+----+-----+-----------------+-----------------+
only showing top 10 rows



In [28]:
rf_prediction.columns

['store_id',
 'dept_id',
 'year',
 'month',
 'sell_price',
 'snap_WI',
 'sales',
 'lag_1',
 'lag_2',
 'lag_3',
 'store_id_index',
 'year_index',
 'features',
 'prediction']

In [29]:
Evaluator_mape = MAPE(predictionCol="prediction", labelCol="sales")

In [30]:
evaluation = Evaluator_mape.evaluate(rf_prediction)

In [31]:
print("Evaluation matrix Score for RFMODEL:", evaluation)

Evaluation matrix Score for RFMODEL: 0.01589810353788771


In [33]:
rf_prediction.toPandas().to_csv('forecast_randomforest_data.csv')

# XGBOOST ESTIMATOR IMPLEMENTATION #

In [34]:
train_df.printSchema()

root
 |-- store_id: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- sell_price: double (nullable = false)
 |-- snap_WI: long (nullable = true)
 |-- sales: double (nullable = false)
 |-- lag_1: double (nullable = false)
 |-- lag_2: double (nullable = false)
 |-- lag_3: double (nullable = false)
 |-- store_id_index: double (nullable = false)
 |-- year_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [35]:
XGBoostModel = XGBoost(inputCols=inputColumns, labelCol="sales").fit(train_df)

training for finding best hyper parameters
mape score: 0.013551008682587906
training for finding best hyper parameters            
mape score:                                           
0.012525305891937562                                  
training for finding best hyper parameters            
mape score:                                                                       
0.04150499665162989                                                               
training for finding best hyper parameters                                        
mape score:                                                                       
0.031568211067719155                                                              
training for finding best hyper parameters                                        
mape score:                                                                       
0.02683252675087867                                                               
training for finding best hyper paramete

In [36]:
pred = XGBoostModel.transform(test_df)

C:\Users\muhammad.kamran01\spark3\python\pyspark\sql\session.py:505: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [37]:
pred.show(12)

+-----+----+-----+-----------------+------------------+
|store|year|month|       prediction|            actual|
+-----+----+-----+-----------------+------------------+
|  0.0| 4.0|    1|7.920180797576904| 7.065613363597717|
|  0.0| 4.0|    2|7.973705291748047|  7.00033446027523|
|  0.0| 4.0|    3| 8.04426383972168| 7.212294468500341|
|  0.0| 4.0|    4|7.989631652832031| 7.355641102974253|
|  0.0| 4.0|    5|7.922354698181152| 7.315218389752975|
|  0.0| 4.0|    6|7.922354698181152| 7.097548850614793|
|  0.0| 4.0|    7|7.922354698181152| 7.123672785204607|
|  0.0| 4.0|    8|7.922354698181152| 7.136483208590247|
|  0.0| 4.0|    9| 7.89372444152832| 6.954638864880987|
|  0.0| 4.0|   10| 7.89372444152832| 7.500529485395295|
|  0.0| 4.0|   11|   7.732666015625| 7.284134806195205|
|  0.0| 4.0|   12| 7.89372444152832|7.2485040723706105|
+-----+----+-----+-----------------+------------------+
only showing top 12 rows



In [38]:
pred.toPandas().to_csv('forecast_xgboost_data.csv')

In [39]:
Evaluator_mape_xgBoost = MAPE(predictionCol="prediction", labelCol="actual")

In [40]:
evaluation_xgb = Evaluator_mape_xgBoost.evaluate(pred)

In [41]:
print("Evaluation matrix Score for XGFMODEL:", evaluation_xgb)

Evaluation matrix Score for XGFMODEL: 0.02676990634890345


In [66]:
antilog_transformer = AntiLog(inputCols=["prediction","sales"])

Saving best model


# Prediction using the best model #

In [ ]:
savedModel = rfModel.load("rfmodel.sav")

In [68]:
pred_best_model = savedModel.transform(test_df)

In [56]:
pred_best_model.select(["store_id","year","month","sales","prediction"]).show(12)

+--------+----+-----+------------------+-----------------+
|store_id|year|month|             sales|       prediction|
+--------+----+-----+------------------+-----------------+
|    CA_1|2015|    1| 7.065613363597717|7.544214479794464|
|    CA_1|2015|    2|  7.00033446027523|6.946529825313822|
|    CA_1|2015|    3| 7.212294468500341| 6.62432624982336|
|    CA_1|2015|    4| 7.355641102974253|7.329366636250969|
|    CA_1|2015|    5| 7.315218389752975|7.366029644624244|
|    CA_1|2015|    6| 7.097548850614793|7.343149957590786|
|    CA_1|2015|    7| 7.123672785204607|7.369674197563143|
|    CA_1|2015|    8| 7.136483208590247|7.412908428746474|
|    CA_1|2015|    9| 6.954638864880987|7.380855663822053|
|    CA_1|2015|   10| 7.500529485395295|6.657383137102154|
|    CA_1|2015|   11| 7.284134806195205|7.419557355947075|
|    CA_1|2015|   12|7.2485040723706105|7.662119457445414|
+--------+----+-----+------------------+-----------------+
only showing top 12 rows

